# Trabalho Final - Rede Aberta de Filas
## Modelagem de Avaliação e Desempenho 2024.2
### Alunas: Melissa Pereira Guarilha, Mellanie Pereira Guarilha

O objetivo do trabalho é pipipo popopo.

## Simulação

a solução idealizada para simulação foi construir um esquema onde existem 2 tipos diferentes de servidores a serem instanciados (servidores de entrada e servidores intermédiarios), cada servidor é responsável por servir um job durante determinado tempo.

Os jobs inicialmente sao criados com uma taxa de chegada que tem distribuição V.A exponencial com média 0,5. Após criados os jobs com seus tempos de chegadas, todos eles são colocados na fila do servidor 1. Logo após os jobs são consumidos de acordo com a disponibilidade do servidor 1 (e se o tempo que eles chegarem for maior que o tempo atual, já que não podem ser servidos antes de "existirem").

A abordagem de adicionar todos os jobs na fila logo em sua chegada foi tomada procurando simplificar o processo de chegada e geração de jobs na fila, já que procuravamos simular o sistema e seu tempo médio.

As distribuições de tempo de serviço foram setadas em diferentes funções para cada servidor.

(escrever mais aqui sobre as escolhas e detalhes do projeto, em suma.)

In [1]:
from typing import Callable, Tuple
import numpy as np
import queue
import random

In [2]:
class Job:
  """
  Define um job.

  Attributes
    ----------
    id : int
        O id do job.
    tempo_de_chegada : int
        O tempo de chegada do job no sistema.
    tempo_de_saida : int
        O tempo de saida do job no sistema.
  """
  def __init__(self, id: int, tempo_de_chegada: float):
    self.id = id
    self.tempo_de_chegada = tempo_de_chegada
    self.tempo_de_saida = 0

In [3]:
def gera_jobs(quantidade_de_jobs: int, warmup: int, tempo_entre_chegadas: float = 0.5) -> list[Job]:
  """
  Gera uma lista de objetos Job com diferentes tempos de chegada.
  Os tempos de chegada seguem uma V.A Exponencial de média 0.5.

    Parameters:
        quantidade_de_jobs (int) : Número de jobs a serem gerados.
        warmup (int) : Número de jobs de warmup a serem gerados.
        tempo_entre_chegadas (float, optional) : Média da variável exponencial (1/lambda). Valor padrão: 0.5.
    Returns:
        list : Uma lista de objetos Job.
  """
  tempo_atual = 0
  jobs = []
  for i in range(0, quantidade_de_jobs + warmup):
    jobs.append(Job(i, tempo_atual))
    tempo_atual += np.random.exponential(tempo_entre_chegadas)
  return jobs

In [4]:
class ServidorInicial():
  """
  <documentação do servidor 1>

  Attributes
    ----------
        tempo_de_serviço : float
          
        ocupado : bool

        tempo_previsto_de_termino : float

        fila : queue.Queue()

        job_trabalhando : Job or None

  """
  def __init__(self, tempo_de_serviço: float):
    self.tempo_de_serviço = tempo_de_serviço
    self.ocupado = False
    self.tempo_previsto_de_termino = 0
    self.fila = queue.Queue()
    self.job_trabalhando = None

  
  def trabalha(self, tempo_atual: float, job: Job = None):
    """
    <doc da funcao> # o servidor começa a executar um job se ele não estiver ocupado. Se estiver ocupado, coloca o job na fila.

      Parameters:
    """
    if not self.ocupado and not self.fila.empty():

      ## condição para não rodar jobs que teoricamente "ainda não chegaram"
      if tempo_atual < self.fila.queue[0].tempo_de_chegada:
        return

      job = self.fila.get(job)
      self.job_trabalhando = job
      job.tempo_de_saida = tempo_atual + self.tempo_de_serviço

      self.ocupado = True

  def conclui_trabalho(self, tempo_atual: float) -> Job:
    """
    <doc da funcao> # conclui o trabalho que o servidor estava trabalhando  

      Parameters:
      Returns:
    """
    if self.ocupado and tempo_atual >= self.job_trabalhando.tempo_de_saida:
      job_saindo = self.job_trabalhando

      self.ocupado = False
      self.job_trabalhando = None

      return job_saindo

In [5]:
class ServidoresFinais():
  """
  <documentação do servidor 1>

  Attributes
    ----------
        tempo_de_serviço : float

        chance_de_retornar : float, optional
          
        ocupado : bool

        tempo_previsto_de_termino : float

        fila : queue.Queue()

  """
  def __init__(self, tempo_de_serviço: float, chance_de_retornar: float = 0):
    self.tempo_de_serviço = tempo_de_serviço
    self.chance_de_retornar = chance_de_retornar
    self.ocupado = False
    self.tempo_previsto_de_termino = 0
    self.fila = queue.Queue()
  
  def trabalha(self, tempo_atual: float):
    """
    <doc da funcao> # o servidor começa a executar um job se ele não estiver ocupado. Se estiver ocupado, coloca o job na fila.

      Parameters:
    """
    job = None

    if not self.ocupado and not self.fila.empty():
      job = self.fila.get(job)
      job.tempo_de_saida = tempo_atual + self.tempo_de_serviço

      self.ocupado = True
      self.tempo_previsto_de_termino = tempo_atual + self.tempo_de_serviço

      ## calcula se o job retornará para fila com probabilidade 'chance_de_retornar'
      if random.random() < self.chance_de_retornar:
        self.fila.put(job)
  
  def conclui_trabalho(self, tempo_atual: float):
    """
    <doc da funcao> # conclui o trabalho que o servidor estava trabalhando  

      Parameters:
    """
    if self.ocupado and tempo_atual >= self.tempo_previsto_de_termino:
      self.ocupado = False
      self.tempo_previsto_de_termino = 0

      self.trabalha(tempo_atual)

In [6]:
def sistema(jobs: list[Job], funcao_servidor_1: Callable[[], float], funcao_servidor_2: Callable[[], float], funcao_servidor_3: Callable[[], float]):
  """
    <doc da funcao>

      Parameters:
  """
  tempo_atual = 0

  Servidor1 = ServidorInicial(funcao_servidor_1())
  Servidor2 = ServidoresFinais(funcao_servidor_2(), 0.2)
  Servidor3 = ServidoresFinais(funcao_servidor_3())

  opcoes = ["S2", "S3"]
  probabilidades = [0.5, 0.5]

  ## insere os jobs na fila do servidor 1
  for job in jobs:
    Servidor1.fila.put(job)

  ## o primeiro job a chegar é sempre executado direto, pq a fila tá vazia
  Servidor1.trabalha(tempo_atual)

  while not Servidor1.fila.empty() or Servidor1.ocupado or not Servidor2.fila.empty() or Servidor2.ocupado or not Servidor3.fila.empty() or Servidor3.ocupado:

    # tenta executar o proximo serviço, se o servidor estiver livre
    job = Servidor1.conclui_trabalho(tempo_atual)
    Servidor1.trabalha(tempo_atual)
    
    # # se o job saiu do servidor 1:
    if job:
      #print(f"o job com id:{job.id} saiu do servidor 1 no tempo: {tempo_atual} - chegada: {job.tempo_de_chegada} - tempo no sistema: {job.tempo_de_saida}")
      # escolhe o servidor com base nas probabilidades
      servidor_escolhido = random.choices(opcoes, probabilidades)[0]

      if servidor_escolhido == "S2":
        #print(f"o job com id:{job.id} escolheu o servidor S2.")
        # adiciona o job na fila do servidor 2
        Servidor2.fila.put(job)
        Servidor2.trabalha(tempo_atual)
      else:
        #print(f"o job com id:{job.id} escolheu o servidor S3.")
        # adiciona o job na fila do servidor 3
        Servidor3.fila.put(job)
        Servidor3.trabalha(tempo_atual)

    Servidor2.conclui_trabalho(tempo_atual)
    Servidor3.conclui_trabalho(tempo_atual)

    tempo_atual += 0.01

abaixo as definições dos tempos de serviço etc

In [7]:
# Situação 1: os tempos de serviço são fixos, determinísticos, e iguais a 0.4s, 0.6s e 0.95s, respectivamente.
def S1_deterministico(): return 0.4
def S2_deterministico(): return 0.6
def S3_deterministico(): return 0.95

# Situação 2: os tempos de serviço nos três servidores são V.A.s uniformes nos intervalos (0.1, 0.7), (0.1, 1.1) e (0.1, 1.8), respectivamente. 
def S1_uniforme(): return np.random.uniform(0.1, 0.7)
def S2_uniforme(): return np.random.uniform(0.1, 1.1)
def S3_uniforme(): return np.random.uniform(0.1, 1.8)

# Situação 3: os tempos de serviço são V.A.s exponenciais com médias 0.4s, 0.6s e 0.95s, respectivamente.
def S1_exponencial(): return np.random.exponential(0.4)
def S2_exponencial(): return np.random.exponential(0.6)
def S3_exponencial(): return np.random.exponential(0.95)

### Execução da simulação

A simulação foi executada utilizando 10000 jobs de warmup e 10000 jobs "reais", para cada situação.

In [8]:
QUANTIDADE_DE_JOBS, WARMUP = 10000, 10000

In [9]:
jobs_deterministicos = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_deterministicos, S1_deterministico, S2_deterministico, S3_deterministico)

In [10]:
jobs_uniformes = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_uniformes, S1_uniforme, S2_uniforme, S3_uniforme)

In [11]:
jobs_exponenciais = gera_jobs(QUANTIDADE_DE_JOBS, WARMUP)
sistema(jobs_exponenciais, S1_exponencial, S2_exponencial, S3_exponencial)

## Métricas

A abordagem utilizada para calcular as métricas foi pegar o tempo de saida do job reduzir do tempo de chegada e utilizar esse valor como "tempo no sistema".
A partir disso foi calculado a média do tempo no sistema e o desvio padrão.

In [12]:
def calcula_metricas(jobs: list[Job]) -> Tuple[float, float]:
  """
  <doc da funcao> # calcula as métricas de tempo medio no sistema e desvio padrao

    Parameters:
  """
  tempo_total_no_sistema = sum((job.tempo_de_saida - job.tempo_de_chegada) for job in jobs[WARMUP:])
  tempo_medio_no_sistema = tempo_total_no_sistema / QUANTIDADE_DE_JOBS

  desvio_padrao_no_tempo_do_sistema = np.std([job.tempo_de_saida - job.tempo_de_chegada for job in jobs[WARMUP:]])

  return tempo_medio_no_sistema, desvio_padrao_no_tempo_do_sistema

Calculando as métricas para cada situação.

In [13]:
metricas_deterministico = calcula_metricas(jobs_deterministicos)
metricas_uniforme = calcula_metricas(jobs_uniformes)
metricas_exponencial = calcula_metricas(jobs_exponenciais)

In [14]:
print(f"Situação 1 (Tempos de serviço determinísticos)\tTempo médio do sistema: {metricas_deterministico[0]}\tDesvio padrão no tempo do sistema: {metricas_deterministico[1]}")
print(f"Situação 2 (Tempos de serviço uniformes)\tTempo médio do sistema: {metricas_uniforme[0]}\tDesvio padrão no tempo do sistema: {metricas_uniforme[1]}")
print(f"Situação 3 (Tempos de serviço exponenciais)\tTempo médio do sistema: {metricas_exponencial[0]}\tDesvio padrão no tempo do sistema: {metricas_exponencial[1]}")

Situação 1 (Tempos de serviço determinísticos)	Tempo médio do sistema: 12.961196477501966	Desvio padrão no tempo do sistema: 15.675379957805404
Situação 2 (Tempos de serviço uniformes)	Tempo médio do sistema: 1.1446386455524236	Desvio padrão no tempo do sistema: 0.6375873594031732
Situação 3 (Tempos de serviço exponenciais)	Tempo médio do sistema: 0.8046111517316822	Desvio padrão no tempo do sistema: 0.492782177353561


Escrever alguma conclusão basica sobre a simulação.